# VarLMM.jl

`VarLMM.jl` is a Julia package for modeling within-subject variability of a longitudinal measurement. 


# Model Details 

In addition to mean levels, it can be important to model factors influencing within-subject variability of longitudinal outcomes. We utilize a modified linear mixed effects model that allows for within-subject variability to be modeled as through covariates. It is based on the [Mixed Effects Multiple Location Scale Model introduced by Dzubar et al. (2020)](https://link.springer.com/article/10.3758/s13428-019-01322-1). 

The procedure assumes the following model for the data:

Data:
- $\textbf{y}_{ij}$ longitudinal response of subject $i$ at time $j$
- $\textbf{x}_{ij}$ mean fixed effects covariates of subject $i$ at time $j$
- $\textbf{z}_{ij}$ random (location) effects covariates of subject $i$ at time $j$
- $\textbf{w}_{ij}$ within-subject variance fixed effects covariates of subject $i$ at time $j$

Parameters:
- $\boldsymbol{\beta}$ mean fixed effects
- $\boldsymbol{\tau}$ within-subject variance fixed effects
- $\boldsymbol{\boldsymbol{\gamma}_i}$ random location effects of subject $i$
- $\boldsymbol{\Sigma}_{\boldsymbol{\gamma}}$ random (location) effects covariance matrix
- $\omega_i$ random scale effect of subject $i$
- $\sigma_\omega^2$ variance of random scale effect
- $\boldsymbol{\Sigma}_{\boldsymbol{\gamma} \omega}$ joint random effects covariance matrix

Other:
- $\mathcal{D(a, b)}$ unspecified distribution with mean $a$ and variance $b$
- $\epsilon_{ij}$ error term capturing within-subject variability



The model:
\begin{eqnarray*}
y_{ij} &=& \textbf{x}_{ij}^T\boldsymbol{\beta} + \textbf{z}_{ij}^T\boldsymbol{\gamma}_i + \epsilon_{ij}, \\
\epsilon_{ij} &\sim& \mathcal{D}(0, \sigma_{\epsilon_{ij}}^2), \\
\boldsymbol{\gamma_i} &=& (\gamma_{i1}, \gamma_{i2}, \cdots, \gamma_{iq})^T \sim \mathcal{D}(\mathbf{0}_{q}, \boldsymbol{\Sigma}_{\boldsymbol{\gamma}}),
\end{eqnarray*}

where

\begin{eqnarray*}
\sigma_{\epsilon_{ij}}^2 = \exp (\textbf{w}_{ij}^T \boldsymbol{\tau} + \boldsymbol{\ell}_{\boldsymbol{\gamma} \omega}^T \boldsymbol{\gamma_i} + \omega_i)  \text{,   }\omega_i \sim \mathcal{D}(0, \sigma_\omega^2)
\end{eqnarray*}

represents the within-subject variance with $\boldsymbol{\ell}_{\gamma \omega}^T$ coming from the Cholesky factor of the covariance matrix of the joint distribution of random effects ($\gamma_i$, $\omega_i$). 
The joint distribution

\begin{eqnarray*}
\begin{pmatrix}
\boldsymbol{\gamma_i} \\ \omega_i
\end{pmatrix} \sim \mathcal{D}(\mathbf{0}_{q+1}, \boldsymbol{\Sigma}_{\boldsymbol{\gamma} \omega})
\end{eqnarray*}

and denote the Cholesky decomposition of the covariance matrix $\Sigma_{\gamma w}$ as

\begin{eqnarray*}
\boldsymbol{\Sigma}_{\boldsymbol{\gamma} \omega} &=& \begin{pmatrix}
\boldsymbol{\Sigma}_{\boldsymbol{\gamma}} & \boldsymbol{\sigma}_{\boldsymbol{\gamma} \omega} \\
\boldsymbol{\sigma}_{\boldsymbol{\gamma} \omega}^T & \sigma_\omega^2
\end{pmatrix} = \textbf{L} \textbf{L}^T, \quad
\textbf{L} = \begin{pmatrix}
\textbf{L}_{\boldsymbol{\gamma}} & \mathbf{0} \\
\boldsymbol{\ell}_{\boldsymbol{\gamma} \omega}^T & \ell_{\omega}
\end{pmatrix},
\end{eqnarray*}

where $\textbf{L}_{\boldsymbol{\gamma}}$ is a $q \times q$ upper triangular matrix with positive diagonal entries and $\ell_{\omega} > 0$. The elements of $\boldsymbol{\Sigma}_{\boldsymbol{\gamma} \omega}$ can be expressed in terms of the Cholesky factors as:


\begin{eqnarray*}
\boldsymbol{\Sigma}_{\boldsymbol{\gamma}} &=& \textbf{L}_{\boldsymbol{\gamma}} \textbf{L}_{\boldsymbol{\gamma}}^T \\ 
\boldsymbol{\sigma}_{\boldsymbol{\gamma} \omega} &=& \textbf{L}_{\boldsymbol{\gamma}} \boldsymbol{\ell}_{\gamma \omega} \\
\sigma_\omega^2 &=& \boldsymbol{\ell}_{\gamma \omega}^T \boldsymbol{\ell}_{\gamma \omega} + \ell_{\omega}^2 
\end{eqnarray*}


Our methodology relaxes the distributional assumptions. We only need that the mean and variance of those distributions hold. 

In their software, they fit the model through maximum likelihood, requiring numerically intensive numerical integration. We have derived a computationally efficient and statistically robust method for obtaining estimates of $\boldsymbol{\beta}, \boldsymbol{\tau}, \text{and}, \boldsymbol{\Sigma_\gamma}$. The mean fixed effects, $\boldsymbol{\beta}$ are estimates via weighted least squares, while the variance components $\boldsymbol{\tau}$ and $\Sigma_\gamma$ are estimated via a generalized method of moments based approach. We do not estimate $\omega_i$ or any association between $\boldsymbol{\gamma}_i$ and $\omega_i$. These are treated as nuissance parameters that get absorbed into the estimation of the intercept of $\boldsymbol{\tau}$ - thus if it is likely there is a random scale/association present in the data, the intercept of $\boldsymbol{\tau}$ and any inference of it will not be accurate. 

# Installation

This package requires Julia v1.0 or later, which can be obtained from https://julialang.org/downloads/ or by building Julia from the sources in the https://github.com/JuliaLang/julia repository.

The package has not yet been registered and must be installed using the repository location. Start Julia and use the ] key to switch to the package manager REPL
```
(v1.1) pkg> add TODO: ADD URL
```
Use the backspace key to return to the Julia REPL.

In [1]:
using VarLMM, CSV, Random, JuliaDB

┌ Info: Precompiling VarLMM [2ff19380-1883-49fc-9d10-450face6b90c]
└ @ Base loading.jl:1273


# Example Usage

The example dataset, `varlmm_exdata.csv`, is contained in `data` folder of the package. It is a simulated datatset with 500 individuals, each having 10 observations. The outcome, systolic blood pressure (sbp), is a function of other covariates. Below we read in the data with the [CSV package](https://juliadata.github.io/CSV.jl) which stores it into a `DataFrame` object. The package can take other data table objects, such as `IndexedTables` generated from reading in data with the [JuliaDB](https://github.com/JuliaData/JuliaDB.jl) package. 

In [2]:
filepath = normpath(joinpath(dirname(pathof(VarLMM)), "../data/"))
df = CSV.read(filepath * "varlmm_exdata.csv")

,id,sbp,agegroup,gender,bmi,meds
,Int64,Float64,Float64,String,Float64,String
1,1,139.586,3.0,Male,23.1336,NoMeds
2,1,141.849,3.0,Male,26.5885,NoMeds
3,1,140.484,3.0,Male,24.8428,NoMeds
4,1,141.134,3.0,Male,24.9289,NoMeds
5,1,145.443,3.0,Male,24.8057,NoMeds
6,1,140.053,3.0,Male,24.1583,NoMeds
7,1,142.1,3.0,Male,25.2543,NoMeds
8,1,143.153,3.0,Male,24.3951,NoMeds
9,1,146.675,3.0,Male,26.1514,NoMeds


## Loading Data Into Model

First we will create a VarLmmModel object from the dataframe.

The `VarLmmModel()` function takes the following arguments: 

- `meanformula`: the formula for the mean fixed effects β (variables in X matrix)
- `reformula`: the formula for the mean random effects γ (variables in Z matrix)
- `wsvarformula`: the formula  for the within-subject variance fixed effects τ (variables in W matrix). 
- `idvar`: the id variable for groupings. 
- `datatable`: the datatable holding all of the data for the model. Can be a DataFrame or various types of tables such as an IndexedTable.

For documentation of the ordinalgwas function, type `?VarLmmModel` in Julia REPL.

We will model sbp as a function of age, gender, and bmi. The following commands fit the following model:

$sbp_{ij} = \beta_0 + \beta_1 \text{agegroup}_{ij} + \beta_2 \text{gender}_{ij} + \beta_3 \text{bmi}_{ij} + \gamma_{i0} + \gamma_{i1}\text{bmi} + \epsilon_{ij}$

$\epsilon_{ij}$ is distributed with mean 0 variance $\sigma^2_{\epsilon_{ij}}$

$\gamma_{i} = (\gamma_{i0}, \gamma_{i1})$ has mean **0** and variance $\Sigma_\gamma$

$\sigma^2_{\epsilon_{ij}} = exp(\tau_0 + \tau_1 \text{agegroup}_{ij} + \tau_2 \text{gender}_{ij} + \tau_3 \text{bmi}_{ij})$

In [3]:
vlmm = VarLmmModel(@formula(sbp ~ 1 + agegroup + gender + bmi + meds), 
    @formula(sbp ~ 1 + bmi), 
    @formula(sbp ~ 1 + agegroup + meds + bmi),
    :id, df);

The `vlmm` object has the appropriate data. We can use the `fit!()` function to fit the model.

## Fitting

The `fit!()` function takes the following arguments:
* `m::VarLmmModel` the model to fit.
* `solver` by default this is Ipopt.IpoptSolver(print_level=5, watchdog_shortened_iter_trigger=3)
* `fittype` by default this is :Hybrid. Performing the Hybrid fit described below. The other options are :Weighted and :Unweighted.
* `weightedruns` by default this is 1.

### Fittypes
The `fit!()` function fits with the following procedures based on `fittype`. The default is `:Hybrid`.

**:Hybrid**

1. Intialize parameters through least squares $(\beta, \tau, L_\gamma)$ for initial starting points.
2. Re-Estimate $(\tau, L_\gamma)$ through the unweighted objective function (method of moments).
3. Re-estimate $\beta$ with weighted least squares using $(\tau, L_\gamma)$ estimates. 
4. For 1:weightedruns
    - 4a. Update working covariance matrix $V_i^{(0)}$ with current estimates of $(\tau, \Sigma_\gamma)$.
    - 4b. Re-estimate $(\tau, \Sigma_\gamma)$ using weighted estimating equations (weighted objective function).
5. Use final estimates and conduct inference.


**:Weighted**
1. Intialize parameters through least squares $(\beta, \tau, \Sigma)$ for initial starting points.
2. Re-estimate $\beta$ with weighted least squares using $(\tau, \Sigma_\gamma)$ estimates. 
3. For 1:weightedruns
    - 3a. Update $V_i^{(0)}$ with current estimates of $(\tau, \Sigma_\gamma)$.
    - 3b. Re-estimate $(\tau, \Sigma_\gamma)$ using weighted estimating equations (new objective function).
5. Use final estimates and conduct inference.


**:Unweighted**
1. Intialize parameters through least squares $(\beta, \tau, L_\gamma)$ for initial starting points.
2. Re-Estimate $(\tau, L_\gamma)$ through the unweighted objective function (method of moments).
3. Re-estimate $\beta$ with weighted least squares using $(\tau, L_\gamma)$ estimates. 
4. Use final estimates and conduct inference.


Generally, the unweighted code will return less precise estimates as it is less statistically efficient.

In [4]:
fit!(vlmm)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

Variance linear mixed model fit by method of moments
Number of individuals/clusters: 500 
Total observations: 5011
 
Fixed-effects parameters:
────────────────────────────────────────────────────────────
                      Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────────
β1: (Intercept)   100.077        0.346109   289.15    <1e-99
β2: agegroup       14.9864       0.063344   236.59    <1e-99
β3: gender: Male   -9.92749      0.100267   -99.01    <1e-99
β4: bmi             0.248811     0.0126703   19.64    <1e-85
β5: meds: OnMeds  -10.115        0.123701   -81.77    <1e-99
τ1: (Intercept)    -2.63253      1.0034      -2.62    0.0087
τ2: agegroup        1.50585      0.056719    26.55    <1e-99
τ3: meds: OnMeds   -0.426159     0.0535526   -7.96    <1e-14
τ4: bmi             0.00434955   0.0472928    0.09    0.9267
──────────────────────────────────────────────────────────── 
Σγ : Random Effects Covariance Matrix
 "γ1: (Intercept)"   2.89

By default, the `fit!()` function fits with the `IPOPT` solver. This can be changed easily by specifying a different solver.

In [5]:
fit!(vlmm, NLopt.NLoptSolver(algorithm = :LN_SBPLX, maxeval = 1000), weightedruns=3)

Variance linear mixed model fit by method of moments
Number of individuals/clusters: 500 
Total observations: 5011
 
Fixed-effects parameters:
────────────────────────────────────────────────────────────
                      Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────────
β1: (Intercept)   100.033        0.361794   276.49    <1e-99
β2: agegroup       14.9881       0.063386   236.46    <1e-99
β3: gender: Male   -9.92802      0.100315   -98.97    <1e-99
β4: bmi             0.250406     0.0133009   18.83    <1e-78
β5: meds: OnMeds  -10.1177       0.124154   -81.49    <1e-99
τ1: (Intercept)    -2.6062       0.947234    -2.75    0.0059
τ2: agegroup        1.5118       0.215558     7.01    <1e-11
τ3: meds: OnMeds   -0.435986     0.073288    -5.95    <1e-8 
τ4: bmi             0.00295938   0.0616024    0.05    0.9617
──────────────────────────────────────────────────────────── 
Σγ : Random Effects Covariance Matrix
 "γ1: (Intercept)"   1.45

In [6]:
using KNITRO
fit!(vlmm, KNITRO.KnitroSolver(outlev=3))


Knitro 12.1.1 STUDENT LICENSE (problem size limit = 300)

Knitro 12.1.1 STUDENT LICENSE (problem size limit = 300)

            Student License
       (NOT FOR COMMERCIAL USE)
         Artelys Knitro 12.1.1

Knitro presolve eliminated 0 variables and 0 constraints.

outlev:                  3
The problem is identified as unconstrained.
Knitro changing algorithm from AUTO to 1.
Knitro changing bar_initpt from AUTO to 3.
Knitro changing bar_murule from AUTO to 4.
Knitro changing bar_penaltycons from AUTO to 0.
Knitro changing bar_penaltyrule from AUTO to 2.
Knitro changing bar_switchrule from AUTO to 0.
Knitro changing linesearch from AUTO to 2.
Knitro changing linsolver from AUTO to 2.

Problem Characteristics                                 (   Presolved)
-----------------------
Objective goal:  Minimize
Objective type:  general
Number of variables:                                  7 (           7)
    bounded below only:                               0 (           0)
    bounded abov

Variance linear mixed model fit by method of moments
Number of individuals/clusters: 500 
Total observations: 5011
 
Fixed-effects parameters:
────────────────────────────────────────────────────────────
                      Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────────
β1: (Intercept)   100.077        0.346157   289.11    <1e-99
β2: agegroup       14.9864       0.0633441  236.59    <1e-99
β3: gender: Male   -9.92749      0.100267   -99.01    <1e-99
β4: bmi             0.248816     0.0126724   19.63    <1e-85
β5: meds: OnMeds  -10.115        0.123703   -81.77    <1e-99
τ1: (Intercept)    -2.63248      1.00316     -2.62    0.0087
τ2: agegroup        1.50585      0.0565414   26.63    <1e-99
τ3: meds: OnMeds   -0.426143     0.0535561   -7.96    <1e-14
τ4: bmi             0.00434703   0.0472604    0.09    0.9267
──────────────────────────────────────────────────────────── 
Σγ : Random Effects Covariance Matrix
 "γ1: (Intercept)"   2.90

### Weighted Only Estimation Procedure

The following demonstrates using only the weighted objective function to fit the model.

In [7]:
fit!(vlmm; fittype=:Weighted, weightedruns=3)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

  36  2.6542513e+04 0.00e+00 1.64e-01  -2.5 5.45e-01    -  1.00e+00 1.00e+00f  1
  37  2.6542513e+04 0.00e+00 1.92e-02  -2.5 3.00e-01    -  1.00e+00 1.00e+00f  1
  38  2.6542513e+04 0.00e+00 5.52e-02  -3.8 7.85e-01    -  1.00e+00 1.00e+00f  1
  39  2.6542513e+04 0.00e+00 6.58e-03  -3.8 5.59e-01    -  1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  40  2.6542513e+04 0.00e+00 4.78e-03  -3.8 9.14e-01    -  1.00e+00 1.00e+00f  1
  41  2.6542513e+04 0.00e+00 7.96e-04  -3.8 9.36e-01    -  1.00e+00 1.00e+00f  1
  42  2.6542513e+04 0.00e+00 1.33e-04  -5.7 9.89e-01    -  1.00e+00 1.00e+00f  1
  43  2.6542513e+04 0.00e+00 1.87e-05  -5.7 9.98e-01    -  1.00e+00 1.00e+00f  1
  44  2.6542513e+04 0.00e+00 2.54e-06  -5.7 1.00e+00    -  1.00e+00 1.00e+00f  1
  45  2.6542513e+04 0.00e+00 3.45e-07  -8.6 1.00e+00    -  1.00e+00 1.00e+00f  1
  46  2.6542513e+04 0.00e+00 4.67e-08  -8.6 1.00e+00    -  1.00e+00 1.00e+00f  1
  47  2.6542513e+04 0.00e+00

Variance linear mixed model fit by method of moments
Number of individuals/clusters: 500 
Total observations: 5011
 
Fixed-effects parameters:
────────────────────────────────────────────────────────────
                      Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────────
β1: (Intercept)   100.274        0.366324   273.73    <1e-99
β2: agegroup       14.9253       0.0884443  168.75    <1e-99
β3: gender: Male   -9.92546      0.11656    -85.15    <1e-99
β4: bmi             0.242992     0.0131289   18.51    <1e-75
β5: meds: OnMeds  -10.0052       0.134983   -74.12    <1e-99
τ1: (Intercept)    -2.64346      0.452472    -5.84    <1e-8 
τ2: agegroup        1.50826      0.025351    59.49    <1e-99
τ3: meds: OnMeds   -0.435792     0.051299    -8.50    <1e-16
τ4: bmi             0.00485069   0.0178461    0.27    0.7858
──────────────────────────────────────────────────────────── 
Σγ : Random Effects Covariance Matrix
 "γ1: (Intercept)"  0.367

## Fitting Tips

- If the `fit!()` function is giving errors, a different solver may remedy the issue. By default, `VarLMM` is using the IPOPT solver, but it can take any solver that supports [MathProgBase.jl](https://github.com/JuliaOpt/MathProgBase.jl). In our experience, [Knitro.jl](https://github.com/JuliaOpt/KNITRO.jl) works the best, but it has only a commercial or academic trial period. 
- The `fit!()` function can also perform poorly if the variance contribution from the random effects heavily outweighs the contribution from the within-subject variance. Standardizing continuous variables can help this immensely. To standardize a variable, a convenient way is to apply the `zscore` function from [StatsBase.jl](https://github.com/JuliaStats/StatsBase.jl) to the variable.
- Using the `:Weighted` option for `fittype` in the `fit!()` function can also work better with non-standardized variables. This performs the optimization using only the weighted objective function. The `:Weighted` option seems to be more robust to overall, and is worth trying if the `:Hybrid` (default) `fittype` returns an optimization issue or gives odd-looking results.
- Increasing the number of `weightedruns` takes more resources but can be useful to get more precise estimates.

## Simulating responses

The `rvarlmm()` and `rvarlmm!()` functions can be used to generate a respone from user-supplied data and parameters. The `rand!()` command can be used to overwrite the response in a VarLmmModel object based on the parameters and optional user-supplied distribution. 

  

The `rand!(m::VarLmmModel; respdist = MvNormal, γωdist = MvNormal, Σγω = [], kwargs...)` function replaces the responses `m.data[i].y` with a simulated response based on:

- The data in the model object's data `X, Z, W` matrices. 
- The parameter values in the model.
- The condistribution distribution of the response given the random effects.
- The distribution of the random effects.
- If simulating from MvTDistribution, you must specify the degrees of freedom via `df = x`.


The `rvarlmm()` takes arrays of matricies of the data in addition to the reponse. It generates a simulated response from the VarLMM model based on:
- `Xs`: array of each clusters `X`: mean fixed effects covariates
- `Zs`: array of each clusters `Z`: random location effects covariates
- `Ws`: array of each clusters `W`: within-subject variance fixed effects covariates
- `β`: mean fixed effects vector
- `τ`: within-subject variance fixed effects vector
- `respdist`: the distribution for response. Default is MvNormal. 
- `Σγ`: random location effects covariance matrix. 
- `Σγω`: joint random location and random scale effects covariance matrix (if generating from full model).
- If simulating from MvTDistribution, you must specify the degrees of freedom via `df = x`.


The `rvarlmm!()` function can be used to generate a simulated response from the VarLMM model based on a datatable and place the generated response into the datatable with the `respname` field. 

Note: **the datatable MUST be ordered by grouping variable for it to generate in the correct order.**
This can be checked via `datatable == sort(datatable, idvar)`. The response is based on:

- `meanformula`: represents the formula for the mean fixed effects `β` (variables in X matrix)
- `reformula`: represents the formula for the mean random effects γ (variables in Z matrix)
- `wsvarformula`: represents the formula for the within-subject variance fixed effects τ (variables in W matrix)
- `idvar`: the id variable for groupings.
- `datatable`: the data table holding all of the data for the model. For this function it **must be in order**.
- `β`: mean fixed effects vector
- `τ`: within-subject variance fixed effects vector
- `respdist`: the distribution for response. Default is MvNormal. 
- `Σγ`: random location effects covariance matrix. 
- `Σγω`: joint random location and random scale effects covariance matrix (if generating from full model)
- `respname`: symbol representing the simulated response variable name.
- If simulating from MvTDistribution, you must specify the degrees of freedom via `df = x`.


For both functions, only one of the Σγ or Σγω matrices have to be specified in order to use the function. Σγ can be used to specify that the generative model will not include a random scale component. It outputs `ys`: an array of reponse `y` that match the order of the data arrays (`Xs, Zs, Ws`).

In [8]:
@show vlmm.data[1].y
Random.seed!(123)
VarLMM.rand!(vlmm; respdist = MvNormal) 
@show vlmm.data[1].y

(vlmm.data[1]).y = [139.58635186701068, 141.84850248386945, 140.48359574389164, 141.13448128282593, 145.44341004850986, 140.05302471176626, 142.1001598920002, 143.1526453898974, 146.6749897477845]
(vlmm.data[1]).y = [143.14843380324785, 141.72450463028116, 140.5427133618829, 145.04329684342204, 142.13615696624117, 142.91572428914196, 141.9299948466158, 141.31164395125165, 140.3906143414521]


9-element Array{Float64,1}:
 143.14843380324785
 141.72450463028116
 140.5427133618829 
 145.04329684342204
 142.13615696624117
 142.91572428914196
 141.9299948466158 
 141.31164395125165
 140.3906143414521 

In [9]:
t = table((id = [1; 1; 2; 3; 3; 3; 4], y = randn(7),
x1 = ones(7), x2 = randn(7), x3 = randn(7), z1 = ones(7),
z2 = randn(7), w1 = ones(7), w2 = randn(7), w3 = randn(7)))
df = DataFrame(t)

f1 = @formula(y ~ 1 + x2 + x3)
f2 = @formula(y ~ 1 + z2)
f3 = @formula(y ~ 1 + w2 + w3)

β = zeros(3)
τ = zeros(3)
Σγ = [1. 0.; 0. 1.]

first(df, 3)

,id,y,x1,x2,x3,z1,z2,w1,w2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1.54783,1.0,0.268209,-0.0571664,1.0,0.222619,1.0,1.17759
2,1,0.365508,1.0,1.17666,-0.204264,1.0,1.0579,1.0,0.431064
3,2,-0.31447,1.0,0.453137,-0.402403,1.0,-1.65662,1.0,-0.216927


In [10]:
rvarlmm!(f1, f2, f3, :id, df, β, τ;
        Σγ = Σγ, respname = :response)
df[!, :response]

7-element Array{Float64,1}:
 -1.1362274345153838 
 -2.0396426949163917 
 -3.0297957994022724 
  0.7967897326514723 
  1.1225160002462085 
  1.2517509664063533 
  0.34088047862482207

Note: JuliaDB's `IndexedTables` elements cannot be mutated. If you use the `rvarlmm!()` function with a JuliaDB table, you must reassign the `datatable` to the output as shown below:

In [11]:
t = rvarlmm!(f1, f2, f3, :id, t, β, τ;
        Σγ = Σγ, respname = :response)

Table with 7 rows, 11 columns:
Columns:
#   colname   type
─────────────────────
1   id        Int64
2   y         Float64
3   x1        Float64
4   x2        Float64
5   x3        Float64
6   z1        Float64
7   z2        Float64
8   w1        Float64
9   w2        Float64
10  w3        Float64
11  response  Float64